In [5]:
import numpy as np
import random as r
import itertools as it
from numba import njit

In [72]:
four = np.fromfile('board_encoding/four.dat', dtype=int).reshape(32760,4)
five = np.fromfile('board_encoding/five.dat', dtype=int).reshape(360360,5)

@njit
def find(row):
    if len(row) == 4:
        for i in range(32760):
            if row[0] == four[i][0] and row[1] == four[i][1] and row[2] == four[i][2] and row[3] == four[i][3]:
                return i
    else:
        for i in range(360360):
            if row[0] == five[i][0] and row[1] == five[i][1] and row[2] == five[i][2] and row[3] == five[i][3] and row[4] == five[i][4]:
                return i

def to_base_62(n):
    digits = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    s = ""
    while n!=0:
        r = n % 62
        s = digits[r] + s
        n = n // 62
    return s

def to_base_10(base_62):
    base_62_arr = [el for el in base_62][::-1]
    digits = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    n = 0
    while len(base_62_arr) != 0:
        n = digits.index(base_62_arr.pop()) + n * 62
    return n

def encode(cardArray):    
    card = cardArray.copy()
    
    for col in range(5):
        card.T[col] -= 15 * col + 1
    
    B = card.T[0]
    I = card.T[1]
    N = np.delete(card.T[2],2)
    G = card.T[3]
    O = card.T[4]
    
    num = 0
    for row in [B, I, N, G, O]:
        num = find(row) + 360360 * num
    
    return to_base_62(num)


def decode(base_62_num):
    card = np.zeros((5,5))
    base_10 = to_base_10(base_62_num)
    for i in range(5):
        num = base_10 % 360360
        if i == 2:
            card[2] = np.insert(four[num],2,-31)
        else:
            card[4 - i] = five[num]
        base_10 = base_10 // 360360
    
    cardArray = card.T
    for col in range(5):
        card[col] += 15 * col + 1
    
    return np.array(card).T

In [73]:
B = r.sample(range(1,16), 5)
I = r.sample(range(16,31), 5)
N = r.sample(range(31,46), 5)
G = r.sample(range(46,61), 5)
O = r.sample(range(61,76), 5)

cardArray = np.array([B, I, N, G, O]).T
cardArray[2][2] = 0

print(cardArray)

e = encode(cardArray)

print(e)

print(to_base_10(e))

print(decode(e))

[[ 9 29 31 49 65]
 [ 3 17 42 47 70]
 [ 6 28  0 52 64]
 [ 2 16 40 51 74]
 [11 22 35 59 62]]
4iK8GqOV8f5AMr3B
3308100000606021129228109611
[[ 9. 29. 31. 49. 65.]
 [ 3. 17. 42. 47. 70.]
 [ 6. 28.  0. 52. 64.]
 [ 2. 16. 40. 51. 74.]
 [11. 22. 35. 59. 62.]]
